In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df_app=pd.read_csv('application_record.csv')
df_cred=pd.read_csv('credit_record.csv')
print(df_app.head(), "\n" , df_cred.head())

        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0  5008804           M            Y               Y             0   
1  5008805           M            Y               Y             0   
2  5008806           M            Y               Y             0   
3  5008808           F            N               Y             0   
4  5008809           F            N               Y             0   

   AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0          427500.0               Working               Higher education   
1          427500.0               Working               Higher education   
2          112500.0               Working  Secondary / secondary special   
3          270000.0  Commercial associate  Secondary / secondary special   
4          270000.0  Commercial associate  Secondary / secondary special   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
0        Civil marriage   Rented apartment      -12005 

In [3]:
df_app.rename(columns={'ID': 'CUST_ID', 'CODE_GENDER': 'GENDER','FLAG_OWN_CAR': 'OWN_CAR',
                       'FLAG_OWN_REALTY': 'PROPERTY', 'CNT_CHILDREN': 'NUM_CHILDREN',
                       'AMT_INCOME_TOTAL': 'INCOME', 'NAME_INCOME_TYPE': 'INCOME_TYPE',
                       'NAME_EDUCATION_TYPE': 'EDUCATION', 'NAME_FAMILY_STATUS': 'FAMILY_STATUS',
                       'NAME_HOUSING_TYPE': 'HOUSING_TYPE', 'DAYS_BIRTH': 'AGE', 'DAYS_EMPLOYED': 'YEARS_EMPLOYED',
                       'FLAG_MOBIL': 'MOBILE_STATUS', 'FLAG_WORK_PHONE': 'WORK_PHONE_STATUS',
                       'FLAG_PHONE': 'PHONE_STATUS', 'FLAG_EMAIL': 'EMAIL_STATUS','OCCUPATION_TYPE': 'OCCUPATION'}, inplace=True)

df_app['AGE']=(-df_app['AGE']/365).astype(int)
df_app["YEARS_EMPLOYED"] = df_app["YEARS_EMPLOYED"].apply(lambda x: np.round(float(-x / 365)) if pd.notnull(x) and x < 0 else 0)
#0 indicates unemployed, other numbers indicate years employed 
df_app.head()

,CUST_ID,GENDER,OWN_CAR,PROPERTY,NUM_CHILDREN,INCOME,INCOME_TYPE,EDUCATION,FAMILY_STATUS,HOUSING_TYPE,AGE,YEARS_EMPLOYED,MOBILE_STATUS,WORK_PHONE_STATUS,PHONE_STATUS,EMAIL_STATUS,OCCUPATION,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32,12.0,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32,12.0,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,58,3.0,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52,8.0,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52,8.0,1,0,1,1,Sales staff,1.0


In [ ]:
#Dealing with duplicates by keeping the first occurrence
df_app = (df_app
          .drop_duplicates(subset='ID', keep='first')
          .reset_index(drop=True)
         )
df_cred = (df_cred
           .drop_duplicates(subset=['ID', 'MONTHS_BALANCE'], keep='first')
           .reset_index(drop=True)
          )

In [15]:
counts = df_app['ID'].value_counts()

if counts.max() > 1:
    print(counts[counts > 1])
else:
    print("No duplicates found!")

ID
7702516    2
7602432    2
7836971    2
7213374    2
7052783    2
7023651    2
7838075    2
7636389    2
7052812    2
7372589    2
7155150    2
7090931    2
7091721    2
7137299    2
7744386    2
7772847    2
7636756    2
7317997    2
7053557    2
7742853    2
7576316    2
7099881    2
7046068    2
7036518    2
7742298    2
7022197    2
7603224    2
7023108    2
7089090    2
7702238    2
7243768    2
7045885    2
7836711    2
7743418    2
7045794    2
7618285    2
7207977    2
7154598    2
7154819    2
7024111    2
7282535    2
7050948    2
7416167    2
7174719    2
7833087    2
7135270    2
7022327    2
Name: count, dtype: int64


In [ ]:
df_app[df_app['CUST_ID']==7089090]#Duplicates in df_app seem to be retired individuals and individuals switching jobs

,CUST_ID,GENDER,OWN_CAR,PROPERTY,NUM_CHILDREN,INCOME,INCOME_TYPE,EDUCATION,FAMILY_STATUS,HOUSING_TYPE,AGE,YEARS_EMPLOYED,MOBILE_STATUS,WORK_PHONE_STATUS,PHONE_STATUS,EMAIL_STATUS,OCCUPATION,CNT_FAM_MEMBERS
425539,7089090,F,N,N,1,180000.0,Working,Secondary / secondary special,Married,House / apartment,38,2.0,1,0,1,0,Laborers,3.0
426198,7089090,F,N,Y,1,189000.0,Commercial associate,Incomplete higher,Married,House / apartment,27,5.0,1,0,0,0,Core staff,3.0


In [17]:
counts_cred = df_cred['ID'].value_counts()

if counts_cred.max() > 1:
    print(counts_cred[counts_cred > 1])
    print(f"Total duplicate IDs in credit record: {len(counts_cred[counts_cred > 1])}")
else:
    print("No duplicates found!")

ID
5001730    61
5002160    61
5002165    61
5002171    61
5002283    61
           ..
5150062     2
5150130     2
5150242     2
5150466     2
5150485     2
Name: count, Length: 45586, dtype: int64
Total duplicate IDs in credit record: 45586


In [ ]:
print(df_cred[df_cred['ID'] == 5001730])
#the duplicate values is a time series so we need to do some feature engineering 

          ID  MONTHS_BALANCE STATUS
377  5001730               0      C
378  5001730              -1      C
379  5001730              -2      C
380  5001730              -3      C
381  5001730              -4      C
..       ...             ...    ...
433  5001730             -56      0
434  5001730             -57      0
435  5001730             -58      0
436  5001730             -59      0
437  5001730             -60      0

[61 rows x 3 columns]


In [3]:
print("Null values in application record", df_app.isnull().sum(),"Null values in credit record", df_cred.isnull().sum())
#Occupation Type has the most null values
#We can either drop the column or impute the values

Null values in application record ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64 Null values in credit record ID                0
MONTHS_BALANCE    0
STATUS            0
dtype: int64


In [4]:
print(df_app.info(), df_cred.info())
df_app['OCCUPATION_TYPE'].value_counts()

<class 'pandas.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  str    
 2   FLAG_OWN_CAR         438557 non-null  str    
 3   FLAG_OWN_REALTY      438557 non-null  str    
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  str    
 7   NAME_EDUCATION_TYPE  438557 non-null  str    
 8   NAME_FAMILY_STATUS   438557 non-null  str    
 9   NAME_HOUSING_TYPE    438557 non-null  str    
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-null  int64  
 15  FLAG_EMAIL           438557 

OCCUPATION_TYPE
Laborers                 78240
Core staff               43007
Sales staff              41098
Managers                 35487
Drivers                  26090
High skill tech staff    17289
Accountants              15985
Medicine staff           13520
Cooking staff             8076
Security staff            7993
Cleaning staff            5845
Private service staff     3456
Low-skill Laborers        2140
Secretaries               2044
Waiters/barmen staff      1665
Realty agents             1041
HR staff                   774
IT staff                   604
Name: count, dtype: int64

In [5]:
avg_income_by_occupation = df_app.groupby('OCCUPATION_TYPE')['AMT_INCOME_TOTAL'].mean().sort_values(ascending=False)
print(avg_income_by_occupation.round(2))

OCCUPATION_TYPE
Managers                 282125.94
Realty agents            223510.81
Accountants              215137.27
IT staff                 210958.73
HR staff                 203620.69
Drivers                  202065.24
Private service staff    201600.65
High skill tech staff    195712.01
Core staff               190905.84
Laborers                 179865.81
Sales staff              170912.76
Security staff           169835.11
Secretaries              169456.88
Medicine staff           168102.20
Waiters/barmen staff     158796.62
Cooking staff            155848.43
Low-skill Laborers       150334.31
Cleaning staff           147064.46
Name: AMT_INCOME_TOTAL, dtype: float64


In [ ]:
# Fill null values in OCCUPATION_TYPE with the occupation that has the closest average income to the person's income
def assign_occupation(row):
    if pd.isna(row['OCCUPATION_TYPE']):
        # Find the occupation with closest average income to this person's income
        closest_occupation = (avg_income_by_occupation - row['AMT_INCOME_TOTAL']).abs().idxmin()
        return closest_occupation
    return row['OCCUPATION_TYPE']

df_app['OCCUPATION_TYPE'] = df_app.apply(assign_occupation, axis=1)
print(f"Null values after imputation: {df_app['OCCUPATION_TYPE'].isnull().sum()}")

In [6]:
df_app.isnull().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64